<a href="https://colab.research.google.com/github/vggls/M.Sc._AI_Thesis/blob/main/experiments/breakhis_resnet50/MaxSensitivity_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38262 sha256=bc02170ce946beb5eb1857cae166e56d8d9a146d6e3b419b49c0f09ab6d38112
  Stored in directory: /root/.cache/pip/wheels/9b/ae/bd/221d138b169c4867b2ae5c893474d0b23bae6e4750bc95bc55
Successfully built grad-cam


In [ ]:
import shutil
import pickle

import torch
from torchvision  import transforms

from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code
from breakhis_slide_allocation import allocate_slides_to_datasets
from breakhis_binary import create_datasets, summaries
from max_sensitivity import sample_eps_Inf, get_explanation, get_exp_sens, MaxSensitivity_Dataset, plot_scores_frequency, scores_boxplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Download from Kaggle**


In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ambarish/breakhis

100% 3.98G/3.99G [03:28<00:00, 20.9MB/s]
100% 3.99G/3.99G [03:28<00:00, 20.5MB/s]


In [ ]:
! unzip breakhis.zip

# **3. Test dataset**

In [ ]:
#shutil.rmtree('./test_dataset')

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
training_dataset, validation_dataset, test_dataset = create_datasets(train_transforms, test_transforms)

No benign slides overlap in training, validation and test sets!
No malignant slides overlap in training, validation and test sets!


In [ ]:
# do not need them in this notebook
shutil.rmtree('./training_dataset')
shutil.rmtree('./validation_dataset')

# **4. Load model**

In [ ]:
model_path = './drive/MyDrive/Colab_Notebooks/dataset_models/BreakHis/Models/'
resnet50 = torch.load(model_path + 'breakhis_resnet50.pt')
resnet50 = resnet50.cuda()
resnet50 = resnet50.eval()

In [ ]:
layers = [resnet50.conv1, resnet50.bn1, resnet50.layer1, resnet50.layer2, resnet50.layer3, resnet50.layer4]
for child in layers:
    for param in child.parameters():
        param.requires_grad = True

# **5. Experiment function**

In [ ]:
def run_experiment(radius, labels, samples_per_img, xai):

    assert xai in ['gradcam', 'hirescam']

    if xai == 'gradcam':
      cam_instance = GradCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
    elif xai == 'hirescam':
      cam_instance = HiResCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
    
    print('radius:{} - labels:{}'.format(radius, labels))

    dataset = [(img, label) for (img, label) in test_dataset if (label in labels)]
    _, data_scores = MaxSensitivity_Dataset(dataset, resnet50, cam_instance, radius, samples_per_img)

    drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/BreakHis/Max_Sensitivity/exp_ResNet50/'

    if xai == 'gradcam':
      with open(drive_path + 'maxsens_resnet50_gradcam_radius_{}_labels_{}.pickle'.format(radius, labels), 'wb') as f:
        pickle.dump(data_scores, f)
    elif xai == 'hirescam':
      with open(drive_path + 'maxsens_resnet50_hirescam_radius_{}_labels_{}.pickle'.format(radius, labels), 'wb') as f:
        pickle.dump(data_scores, f)
    
    return data_scores

# **6. GradCAM**

## **radius 0.05**

In [ ]:
data_scores = run_experiment(radius=0.05, labels=[0,1], samples_per_img=20, xai='gradcam')

radius:0.05 - labels:[0, 1]
Total time: 778.1855580806732 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.59


In [ ]:
sum(data_scores)

53358.25596809387

## **radius 0.1**

In [ ]:
data_scores = run_experiment(radius=0.1, labels=[0,1], samples_per_img=20, xai='gradcam')

radius:0.1 - labels:[0, 1]
Total time: 782.8284747600555 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.59


In [ ]:
sum(data_scores)

76658.79347801208

## **radius 0.2**

In [ ]:
data_scores = run_experiment(radius=0.2, labels=[0,1], samples_per_img=30, xai='gradcam')

radius:0.2 - labels:[0, 1]
Total time: 1169.1989476680756 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.89


In [ ]:
sum(data_scores)

96675.95106124878

## **radius 0.3**

In [ ]:
data_scores = run_experiment(radius=0.3, labels=[0,1], samples_per_img=30, xai='gradcam')

radius:0.3 - labels:[0, 1]
Total time: 1165.541390657425 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.89


In [ ]:
sum(data_scores)

102385.1095199585

## **radius 0.4**

In [ ]:
data_scores = run_experiment(radius=0.4, labels=[0,1], samples_per_img=40, xai='gradcam')

radius:0.4 - labels:[0, 1]
Total time: 1544.7040603160858 secs
Correctly predicted images: 1316/1557
Avg secs per image:  1.17


In [ ]:
sum(data_scores)

106474.06798171997

## **radius 0.5**

In [ ]:
data_scores = run_experiment(radius=0.5, labels=[0,1], samples_per_img=40, xai='gradcam')

radius:0.5 - labels:[0, 1]
Total time: 1528.7787430286407 secs
Correctly predicted images: 1316/1557
Avg secs per image:  1.16


In [ ]:
sum(data_scores)

108337.03437805176

# **7. HiResCAM**

## **radius 0.05**

In [ ]:
data_scores = run_experiment(radius=0.05, labels=[0,1], samples_per_img=20, xai='hirescam')

radius:0.05 - labels:[0, 1]
Total time: 775.6268167495728 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.59


In [ ]:
sum(data_scores)

48041.11091423035

## **radius 0.1**

In [ ]:
data_scores = run_experiment(radius=0.1, labels=[0,1], samples_per_img=20, xai='hirescam')  

radius:0.1 - labels:[0, 1]
Total time: 785.767427444458 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.6


In [ ]:
sum(data_scores)

67949.15669345856

## **radius 0.2**

In [ ]:
data_scores = run_experiment(radius=0.2, labels=[0,1], samples_per_img=30, xai='hirescam')

radius:0.2 - labels:[0, 1]
Total time: 1149.916580915451 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.87


In [ ]:
sum(data_scores)

84808.04773712158

## **radius 0.3**

In [ ]:
data_scores = run_experiment(radius=0.3, labels=[0,1], samples_per_img=30, xai='hirescam')

radius:0.3 - labels:[0, 1]
Total time: 1130.640930891037 secs
Correctly predicted images: 1316/1557
Avg secs per image:  0.86


In [ ]:
sum(data_scores)

90099.30974960327

## **radius 0.4**

In [ ]:
data_scores = run_experiment(radius=0.4, labels=[0,1], samples_per_img=40, xai='hirescam')

radius:0.4 - labels:[0, 1]
Total time: 1501.842985868454 secs
Correctly predicted images: 1316/1557
Avg secs per image:  1.14


In [ ]:
sum(data_scores)

93695.94694519043

## **radius 0.5**

In [ ]:
data_scores = run_experiment(radius=0.5, labels=[0,1], samples_per_img=40, xai='hirescam')

radius:0.5 - labels:[0, 1]
Total time: 1522.7322900295258 secs
Correctly predicted images: 1316/1557
Avg secs per image:  1.16


In [ ]:
sum(data_scores)

95304.00875473022